In [1]:
import pandas as pd
import requests
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)
from datetime import datetime, timedelta
import random
import yfinance as yf
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
import torch
model_id = "deepseek-ai/deepseek-llm-7b-chat"
torch.cuda.set_device(3)  # Sets default to GPU 0
device=torch.device("cuda:3")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map={"": 3},             # auto-distributes across GPUs
    torch_dtype="auto",            # picks bf16 or fp16 depending on availability
)

/home/f20222001/test-venv/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.25it/s]


In [ ]:
class fundamental_analyst:

    def __init__ (self, ticker):
        self.ticker=ticker
    
    def get_financial_info(self):
        ticker = yf.Ticker(self.ticker)
        self.income_stmt = ticker.financials
        self.balance_sheet = ticker.balance_sheet
        self.cash_flow = ticker.cashflow
        self.info = ticker.info
        self.year=self.income_stmt.columns[0]
        self.EPS=self.income_stmt[self.year]["Diluted EPS"]
        self.Net_Income=self.income_stmt[self.year]["Net Income"]
        self.Gross_Profit=self.income_stmt[self.year]["Gross Profit"]
        self.Revenue=self.income_stmt[self.year]["Total Revenue"]
        self.Total_Assets=self.balance_sheet[self.year]["Total Assets"]
        self.Total_Liabilities=self.balance_sheet[self.year]["Total Liabilities Net Minority Interest"]
        self.Shareholders_Equity=self.balance_sheet[self.year]["Stockholders Equity"]
        self.Free_Cash_Flow=self.cash_flow[self.year]["Free Cash Flow"]
        self.Investing_Cash_Flow=self.cash_flow[self.year]["Investing Cash Flow"]
        self.Financing_Cash_Flow=self.cash_flow[self.year]["Financing Cash Flow"]
        self.Operating_Cash_Flow=self.cash_flow[self.year]["Operating Cash Flow"]
        self.P_E=ticker.info['trailingPE']
        self.ROA=ticker.info['returnOnAssets']
        self.ROE=ticker.info['returnOnEquity']
    
    def generate_prompt(self):
        prompt = f"""
You are a fundamental investment analyst. Analyze the financial performance of {self.ticker} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
Do not add assumptions about company operations, reputation, or strategy.

Financials for {self.ticker}:

Income Statement:
Revenue: ${self.Revenue:,.0f}
Gross Profit: ${self.Gross_Profit:,.0f}
Net Income: ${self.Net_Income:,.0f}
EPS (Diluted): {self.EPS:.2f}

Balance Sheet:
Total Assets: ${self.Total_Assets:,.0f}
Total Liabilities: ${self.Total_Liabilities:,.0f}
Shareholders' Equity: ${self.Shareholders_Equity:,.0f}

Cash Flow:
Operating Cash Flow: ${self.Operating_Cash_Flow:,.0f}
Free Cash Flow: ${self.Free_Cash_Flow:,.0f}
Investing Cash Flow: ${self.Investing_Cash_Flow:,.0f}
Financing Cash Flow: ${self.Financing_Cash_Flow:,.0f}

Valuation and Ratios:
P/E Ratio: {self.P_E:.2f}
ROA: {self.ROA:.2%}
ROE: {self.ROE:.2%}

Based on this, what is your investment recommendation? Pick one action candidate."""
        return prompt

In [24]:
prompt=get_financial_info("AAPL")
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

outputs = generator(
    prompt,
    max_new_tokens=500,         # Reduced for memory efficiency
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
    # Memory efficient generation settings
    num_beams=1,                # No beam search to save memory
    #early_stopping=True,
    use_cache=True
)

generated_text = outputs[0]['generated_text']
print(generated_text)

Device set to use cuda:3



        You are a fundamental investment analyst. Analyze the financial performance of AAPL and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
        Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
        Do not add assumptions about company operations, reputation, or strategy.


        Financials for AAPL:

        Income Statement:
        - Revenue: $391,035,000,000
        - Gross Profit: $180,683,000,000
        - Net Income: $93,736,000,000
        - EPS (Diluted): 6.08

        Balance Sheet:
        - Total Assets: $364,980,000,000
        - Total Liabilities: $308,030,000,000
        - Shareholders' Equity: $56,950,000,000

        Cash Flow:
        - Operating Cash Flow: $118,254,000,000
        - Free Cash Flow: $108,807,000,000
        - Investing Cash Flow: $2,935,000,000
        - Financing Cash Flow: $-121,983,000,000

        Valuation and Ratios

In [3]:
class technical_analyst:

    def __init__(self, ticker):
        self.ticker=ticker
    
    def generate_df(self):
        self.data=yf.download(self.ticker, period='1y')

    def compute_rsi(close, period=14):
        delta = close.diff()

        gain = delta.clip(lower=0)
        loss = -delta.clip(upper=0)

        avg_gain = gain.rolling(window=period).mean()
        avg_loss = loss.rolling(window=period).mean()

        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))

        return rsi

    def compute_obv(close, volume):
        direction = close.diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
        obv = (volume * direction).fillna(0).cumsum()
        return obv


In [27]:
data['Close']['AAPL']
temp=pd.DataFrame()
multi_data=[]
for x in tickers:
    temp=pd.DataFrame()
    temp["SMA_5"] = data["Close"][x].rolling(5).mean()
    temp["SMA_15"] = data["Close"][x].rolling(15).mean()
    temp["SMA_50"] = data["Close"][x].rolling(50).mean()

    temp['EMA_5'] = data['Close'][x].ewm(span=5).mean()
    temp['EMA_10'] = data['Close'][x].ewm(span=10).mean()
    temp['EMA_50'] = data['Close'][x].ewm(span=50).mean()
    temp["Date"] = data['Close'].index
    temp["Ticker"] = x
    temp["RSI"]=compute_rsi(data['Close'][x])
    temp["OBV"]=compute_obv(data['Close'][x], data['Volume'][x])
    multi_data.append(temp)
indicators=pd.concat(multi_data)
indicators.set_index(["Ticker", "Date"], inplace=True)
indicators

SMA_5      SMA_15      SMA_50       EMA_5      EMA_10  \
Ticker Date                                                                     
AAPL   2024-07-19         NaN         NaN         NaN  223.265839  223.265839   
       2024-07-22         NaN         NaN         NaN  223.056833  223.074250   
       2024-07-23         NaN         NaN         NaN  223.485874  223.431355   
       2024-07-24         NaN         NaN         NaN  221.008865  221.484718   
       2024-07-25  220.829248         NaN         NaN  219.269381  220.047315   
...                       ...         ...         ...         ...         ...   
NVDA   2025-07-14  163.194003  157.416001  140.625915  162.965192  160.260295   
       2025-07-15  165.334003  159.184668  141.807870  165.543460  162.158423   
       2025-07-16  167.032001  160.749335  142.945429  167.485639  163.833254   
       2025-07-17  168.812000  161.995335  144.129187  169.323759  165.499935   
       2025-07-18  170.310001  163.154668  145.306745  170.352507  166.756311   

                       EMA_50        RSI         OBV  
Ticker Date                                           
AAPL   2024-07-19  223.265839        NaN           0  
       2024-07-22  223.088184        NaN   -48201800  
       2024-07-23  223.391386        NaN    -8241500  
       2024-07-24  221.835036        NaN   -70019100  
       2024-07-25  220.676154        NaN  -121410300  
...                       ...        ...         ...  
NVDA   2025-07-14  143.360036  79.987979  2632260400  
       2025-07-15  144.432246  81.596482  2862887800  
       2025-07-16  145.488681  78.114728  3021719300  
       2025-07-17  146.567607  78.758823  3182560400  
       2025-07-18  147.581080  75.171740  3036452800  

[1000 rows x 8 columns]

In [28]:
def create_technical_prompt(df,t):
    latest = df.loc[t].iloc[-1]
    prompt = f"""
    You are a technical investment analyst. Analyze the recent technical performance of {t} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
    Justify your decision in 4–6 bullet points using technical analysis. Only use the numerical data given. 
    Do not add assumptions about company fundamentals, operations, or strategy.

    Technical Indicators for {t} (most recent data point):

    - SMA 5: {latest['SMA_5']:.2f}
    - SMA 15: {latest['SMA_15']:.2f}
    - SMA 50: {latest['SMA_50']:.2f}

    - EMA 5: {latest['EMA_5']:.2f}
    - EMA 10: {latest['EMA_10']:.2f}
    - EMA 50: {latest['EMA_50']:.2f}

    - RSI: {latest['RSI']:.2f}
    - OBV: {latest['OBV']:,.0f}

    Based on this, what is your investment recommendation? Pick one action candidate.
    """
    return prompt


In [29]:
ta_prompt=create_technical_prompt(indicators,"AAPL")
ta_prompt

'\n    You are a technical investment analyst. Analyze the recent technical performance of AAPL and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. \n    Justify your decision in 4–6 bullet points using technical analysis. Only use the numerical data given. \n    Do not add assumptions about company fundamentals, operations, or strategy.\n\n    Technical Indicators for AAPL (most recent data point):\n\n    - SMA 5: 209.82\n    - SMA 15: 209.59\n    - SMA 50: 204.34\n\n    - EMA 5: 210.37\n    - EMA 10: 209.75\n    - EMA 50: 206.55\n\n    - RSI: 70.07\n    - OBV: 551,218,300\n\n    Based on this, what is your investment recommendation? Pick one action candidate.\n    '

In [30]:
prompt=create_technical_prompt(indicators,"AAPL")
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

outputs = generator(
    prompt,
    max_new_tokens=500,         # Reduced for memory efficiency
    do_sample=True,
    temperature=0.4,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
    # Memory efficient generation settings
    num_beams=1,                # No beam search to save memory
    #early_stopping=True,
    use_cache=True
)

generated_text = outputs[0]['generated_text']
print(generated_text)

Device set to use cuda:3



    You are a technical investment analyst. Analyze the recent technical performance of AAPL and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
    Justify your decision in 4–6 bullet points using technical analysis. Only use the numerical data given. 
    Do not add assumptions about company fundamentals, operations, or strategy.

    Technical Indicators for AAPL (most recent data point):

    - SMA 5: 209.82
    - SMA 15: 209.59
    - SMA 50: 204.34

    - EMA 5: 210.37
    - EMA 10: 209.75
    - EMA 50: 206.55

    - RSI: 70.07
    - OBV: 551,218,300

    Based on this, what is your investment recommendation? Pick one action candidate.
    
```python
# Define technical indicators
sma_5 = 209.82
sma_15 = 209.59
sma_50 = 204.34

ema_5 = 210.37
ema_10 = 209.75
ema_50 = 206.55

rsi = 70.07
obv = 551218300

# Analyze technical performance
# 1. The 5-day simple moving average (SMA) is above the 15-day SMA, indicating an uptrend.
# 2. The 5-day exponential moving average 

In [61]:
def get_news_analysis(t):
    to_date = datetime.today().date()

    from_date = to_date - timedelta(days=7)

    # Format dates as YYYY-MM-DD
    from_str = from_date.strftime('%Y-%m-%d')
    to_str = to_date.strftime('%Y-%m-%d')

    # Construct URL
    symbol = "AAPL"
    api_key = "d1l719pr01qt4thec1pgd1l719pr01qt4thec1q0"
    url = f"https://finnhub.io/api/v1/company-news?symbol={symbol}&from={from_str}&to={to_str}&token={api_key}"
    response = requests.get(url)
    news = response.json()
    filtered_news = [
        item for item in news
        if "apple" in item['headline'].lower()
    ]
    selected_news = random.sample(filtered_news, k=min(10, len(filtered_news)))
    prompt = f"""
You are a sentiment and headlines investment analyst. Analyze the recent technical performance of {t} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
Justify your decision in 4–6 bullet points using sentiment analysis. Only use the headlines given. 
Do not add assumptions about company fundamentals, operations, or strategy.
Headlines:
"""
    for i, item in enumerate(selected_news):
        prompt+=item['headline']
        prompt+='\n'
    prompt+="Based on this, what is your investment recommendation? Pick one action candidate."
    return prompt
    

In [62]:
print(get_news_analysis('AAPL'))


You are a sentiment and headlines investment analyst. Analyze the recent technical performance of AAPL and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
Justify your decision in 4–6 bullet points using sentiment analysis. Only use the headlines given. 
Do not add assumptions about company fundamentals, operations, or strategy.
Headlines:
Top Stock Reports for Apple, JPMorgan Chase & Visa
Apple announces $500 million deal with MP materials
Alphabet vs. Apple: Which Consumer AI Tech Stock is a Better Buy Now?
MP Materials and Apple Announce $500 Million Partnership to Produce Recycled Rare Earth Magnets in the United States
AAPL Investors Have Opportunity to Lead Apple Inc. Securities Fraud Lawsuit
Apple inks $500-million deal with MP Materials for rare earth magnets
Why Apple Fell 18.1% in the First Half of 2025
Apple lands record-breaking 81 Emmy Award nominations with “Severance” leading as this year’s most-nominated series and “The Studio” becoming the most-nominate